# __Bid data platforms final project - Data analysis__

## Imports

In [27]:
import findspark
findspark.init()

import sqlite3
import random
import os
import numpy as np
import pandas as pd
import pyspark
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
import pyspark.pandas as ps
ps.set_option('compute.ops_on_diff_frames', True)

%matplotlib widget
sns.set_style("darkgrid")

import warnings
warnings.filterwarnings('ignore')

In [28]:
DEFAULT_DB_NAME = "NBA_DATA_BIG_DATA_PROJECT.db"

conn = sqlite3.connect('NBA_DATA_BIG_DATA_PROJECT.db')
cursor = conn.cursor()

# Getting all tables from sqlite_master
query = """SELECT name FROM sqlite_master
           WHERE type='table';"""

# executing our sql query
cursor.execute(query)
print("List of tables\n")

# printing all tables list
print(cursor.fetchall())

List of tables

[('Games',), ('Players',), ('Teams',), ('GameLineUps',), ('GameTeamScores',), ('GamePlayerScores',)]


In [29]:
from pyspark.conf import SparkConf

conf = SparkConf()  # create the configuration
conf.set("spark.jars", "/Users/ronibendom/Master/Big Data Platforms/BigDataProject/sqlite-jdbc-3.40.1.0.jar")  # set the spark.jars

...
spark = SparkSession.builder \
        .config(conf=conf) \
        .master("local") \
        .appName("SQLite JDBC")\
        .config("spark.driver.extraClassPath","/Users/ronibendom/Master/Big Data Platforms/BigDataProject/ssqlite-jdbc-3.40.1.0.jar")\
        .config("spark.executor.extraClassPath","/Users/ronibendom/Master/Big Data Platforms/BigDataProject/sqlite-jdbc-3.40.1.0.jar")\
        .getOrCreate()

23/02/05 19:45:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [31]:
df = ps.read_sql("Games", con="jdbc:sqlite:/Users/ronibendom/Master/Big Data Platforms/BigDataProject/NBA_DATA_BIG_DATA_PROJECT.db")
df

Py4JJavaError: An error occurred while calling o262.load.
: java.sql.SQLException: No suitable driver
	at java.sql/java.sql.DriverManager.getDriver(DriverManager.java:300)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$2(JDBCOptions.scala:107)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:107)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:39)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:34)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:350)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:228)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:210)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader

23/02/06 06:35:22 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 943323 ms exceeds timeout 120000 ms
23/02/06 06:35:22 WARN SparkContext: Killing executors is not supported by current scheduler.


In [ ]:
spark.stop()

## Utils

In [ ]:
figure_count = 0

In [ ]:
def index_to_strings(argument, team_name, stat_name, value):
    switcher = {
        0: f'Team with highest {stat_name} is {team_name} - {value}',
        1: f'Team with 2nd highest {stat_name} is {team_name} - {value}',
        2: f'Team with 3rd highest {stat_name} is {team_name} - {value}',
    }

    return switcher.get(argument, "nothing")

In [ ]:
def index_to_strings_players(argument, player_name, stat_name, value):
    switcher = {
        0: f'Player with highest {stat_name} is {player_name} - {value}',
        1: f'Player with 2nd highest {stat_name} is {player_name} - {value}',
        2: f'Player with 3rd highest {stat_name} is {player_name} - {value}',
    }

    return switcher.get(argument, "nothing")

## Read data

In [ ]:
spark = SparkSession.builder.appName('treatmeant').getOrCreate()

In [ ]:
# Use Spark to read the training csv file.
teams = spark.read.csv("Teams.csv", inferSchema=True, header=True)
teams = teams.to_pandas_on_spark()
players = spark.read.csv("Players.csv", inferSchema=True, header=True)
players = players.to_pandas_on_spark()
games = spark.read.csv("Games.csv", inferSchema=True, header=True)
games = games.to_pandas_on_spark()
team_rosters = spark.read.csv("TeamRosters.csv", inferSchema=True, header=True)
team_rosters = team_rosters.to_pandas_on_spark()
game_line_up = spark.read.csv("GameLineUp.csv", inferSchema=True, header=True)
game_line_up = game_line_up.to_pandas_on_spark()
game_team_score = spark.read.csv("GameTeamScore.csv", inferSchema=True, header=True)
game_team_score = game_team_score.to_pandas_on_spark()
game_player_score = spark.read.csv("GamePlayerScore.csv", inferSchema=True, header=True)
game_player_score = game_player_score.to_pandas_on_spark()

## Display data info

In [ ]:
teams_dict = {row['ID']: row['City'] + " " + row['TeamName'] for index, row in teams.iterrows()}
teams.info()

In [ ]:
teams

In [ ]:
players_dict = {row['ID']: row['FirstName'] + " " + row['LastName'] for index, row in players.iterrows()}
players.info()

In [ ]:
games = games.loc[(games['HomeTeamID'].isin(teams_dict.keys())) & (games['VisitorTeamID'].isin(teams_dict.keys())), :]
games.info()

In [ ]:
team_rosters.info()

In [ ]:
game_line_up = game_line_up.loc[game_line_up['GameID'].isin(games['ID'].to_numpy())]
game_line_up.info()

In [ ]:
game_team_score = game_team_score.loc[game_team_score['TeamID'].isin(teams_dict.keys()), :]
game_team_score.info()

In [ ]:
game_player_score = game_player_score.loc[game_player_score['PlayerID'].isin(players_dict.keys()), :]
game_player_score.info()

## __Season level analysis__

### Per game Per group statistics - from each game 2 values are taken into acount, one for each group

__Histogram of number of games a group played in the current season (2022-2023) so far__

In [ ]:
finished_games = games.loc[games['LiveStatus'] == 'Final', :]

In [ ]:
home_games = finished_games['HomeTeamID'].value_counts()
visitor_games = finished_games['VisitorTeamID'].value_counts()
indexes = home_games.index
total_games = pyspark.pandas.Series({teamID: home_games[teamID] + visitor_games[teamID] for teamID in home_games.index.to_list()})

In [ ]:
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(total_games.to_numpy(), alpha = 0.5, bins = np.arange(44, 51))
ax.set(title = "# of games a team played so far in the season", xlabel = '# of games', ylabel='count[teams]')
plt.legend([],[], frameon = False)
plt.show(fig);

__Histogram of number of players per game (for a single group) in the current season (2022-2023) so far__

In [ ]:
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(game_line_up.groupby(['GameID', 'TeamID']).count().to_numpy(), alpha = 0.5)
ax.set(title = "# of players in a single game (per team)", xlabel = '# of players', ylabel='count[games]')
plt.legend([],[], frameon = False)
plt.show(fig);

### Field goals

__Histogram of field goals made out of field goals attempted in a single game per group in the current season (2022-2023) so far__

In [ ]:
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(game_team_score['FGPER'].to_numpy(), alpha = 0.5, kde = True, bins = np.arange(30, 70))
ax.set(title = "Percentage of field goals made in a single game (per team)", xlabel = '% of field goals made', ylabel='count[games]')
plt.legend([],[], frameon = False)
plt.show(fig);

In [ ]:
three_best_teams = game_team_score.loc[:, ['TeamID', 'FGPER']].groupby(['TeamID']).mean().nlargest(3, columns='FGPER')
for i, (index, team) in enumerate(three_best_teams.iterrows()):
    print(index_to_strings(i, teams_dict[index], 'FG%',  f'{team.values[0]:.2f}%'))


### Three points field goals

__Histogram of three point field goal made out of attempted in a single game per group in the current season (2022-2023) so far__

In [ ]:
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(game_team_score['ThreePPER'].to_numpy(), alpha = 0.5, kde = True, bins = np.arange(10, 70))
ax.set(title = "Percentage of three points field goals made in a single game (per team)", xlabel = '% of three points field goals made', ylabel='count[games]')
plt.legend([],[], frameon = False)
plt.show(fig);

In [ ]:
three_best_teams = game_team_score.loc[:, ['TeamID', 'ThreePPER']].groupby(['TeamID']).mean().nlargest(3, columns='ThreePPER')
for i, (index, team) in enumerate(three_best_teams.iterrows()):
    print(index_to_strings(i, teams_dict[index], '3FG%',  f'{team.values[0]:.2f}%'))


In [ ]:
game_player_score.shape

In [ ]:
grouped = game_player_score.to_pandas().loc[:, ['PlayerID', 'ThreePA', 'ThreePPER']].groupby('PlayerID')
grouped.filter(lambda x: x['ThreePA'].count() > 10).loc[:, ['PlayerID', 'ThreePPER']].groupby('PlayerID').mean()

In [ ]:
grouped = game_player_score.to_pandas().loc[:, ['PlayerID', 'ThreePA', 'ThreePPER']].groupby('PlayerID')
three_best_players = grouped.filter(lambda x: x['ThreePA'].count() > 10).loc[:, ['PlayerID', 'ThreePPER']].groupby('PlayerID').mean().nlargest(3, columns='ThreePPER')
for i, (index, player) in enumerate(three_best_players.iterrows()):
    print(index_to_strings_players(i, players_dict[index], '3FG%', f'{player.values[0]:.2f}%'))

__Histogram of three point field goal made out of field goal made in a single game per group in the current season (2022-2023) so far__

In [ ]:
game_team_score['3FGM_to_FGM'] = game_team_score['ThreePM'] * 100 /game_team_score['FGM']
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(game_team_score['3FGM_to_FGM'].to_numpy(), alpha = 0.5, kde = True, bins = np.arange(10, 70))
ax.set(title = "Percentage of 3FGM out og FGM in a single game (per team)", xlabel = '% of 3FGM out of FGM', ylabel='count[games]')
plt.legend([],[], frameon = False)
plt.show(fig);

In [ ]:
three_best_teams = game_team_score.loc[:, ['TeamID', '3FGM_to_FGM']].groupby(['TeamID']).mean().nlargest(3, columns='3FGM_to_FGM')
for i, (index, team) in enumerate(three_best_teams.iterrows()):
    print(index_to_strings(i, teams_dict[index], 'precentage of 3FGM out of FGM', f'{team.values[0]:.2f}%'))

### Free throws

__Histogram of free throws made out of free throws attempted in a single game per group in the current season (2022-2023) so far__

In [ ]:
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(game_team_score['FTPER'].to_numpy(), alpha = 0.5, kde = True)
ax.set(title = "Percentage of free throws made in a single game (per team)", xlabel = '% of free throws made', ylabel='count[games]')
plt.legend([],[], frameon = False)
plt.show(fig);

In [ ]:
three_best_teams = game_team_score.loc[:, ['TeamID', 'FTPER']].groupby(['TeamID']).mean().nlargest(3, columns='FTPER')
for i, (index, team) in enumerate(three_best_teams.iterrows()):
    print(index_to_strings(i, teams_dict[index], 'FT%', f'{team.values[0]:.2f}%'))

### Rebounds

__Histogram of total rebounds in a single game per group in the current season (2022-2023) so far__

In [ ]:
game_team_score['REB'] = game_team_score['OREB'] + game_team_score['DREB']
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(game_team_score['REB'].to_numpy(), alpha = 0.5, kde = True, bins = np.arange(23, 70))
ax.set(title = "# of rebounds in a single game (per team)", xlabel = '# of rebounds', ylabel='count[games]')
plt.legend([],[], frameon = False)
plt.show(fig);

In [ ]:
three_best_teams = game_team_score.loc[:, ['TeamID', 'REB']].groupby(['TeamID']).mean().nlargest(3, columns='REB')
for i, (index, team) in enumerate(three_best_teams.iterrows()):
    print(index_to_strings(i, teams_dict[index], 'avg number of rebounds', f'{team.values[0]:.2f}'))

In [ ]:
game_player_score['REB'] = game_player_score['OREB'] + game_player_score['DREB']
three_best_players = game_player_score.loc[:, ['PlayerID', 'REB']].groupby(['PlayerID']).mean().nlargest(3, columns='REB')
for i, (index, player) in enumerate(three_best_players.iterrows()):
    print(index_to_strings_players(i, players_dict[index], 'avg number of rebounds', f'{player.values[0]:.2f}'))

__Histogram of offensive rebounds in a single game per group in the current season (2022-2023) so far__

In [ ]:
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(game_team_score['OREB'].to_numpy(), alpha = 0.5, kde = True, bins = np.arange(0, 30))
ax.set(title = "# of offensive rebounds in a single game (per team)", xlabel = '# of offensive rebounds', ylabel='count[games]')
plt.legend([],[], frameon = False)
plt.show(fig);

In [ ]:
three_best_teams = game_team_score.loc[:, ['TeamID', 'OREB']].groupby(['TeamID']).mean().nlargest(3, columns='OREB')
for i, (index, team) in enumerate(three_best_teams.iterrows()):
    print(index_to_strings(i, teams_dict[index], 'avg number of offensive rebounds', f'{team.values[0]:.2f}'))

In [ ]:
three_best_players = game_player_score.loc[:, ['PlayerID', 'OREB']].groupby(['PlayerID']).mean().nlargest(3, columns='OREB')
for i, (index, player) in enumerate(three_best_players.iterrows()):
    print(index_to_strings_players(i, players_dict[index], 'avg number of offensive rebounds', f'{player.values[0]:.2f}'))

__Histogram of defensive rebounds in a single game per group in the current season (2022-2023) so far__

In [ ]:
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(game_team_score['DREB'].to_numpy(), alpha = 0.5, kde = True)
ax.set(title = "# of defensive rebounds in a single game (per team)", xlabel = '# of defensive rebounds', ylabel='count[games]')
plt.legend([],[], frameon = False)
plt.show(fig);

In [ ]:
three_best_teams = game_team_score.loc[:, ['TeamID', 'DREB']].groupby(['TeamID']).mean().nlargest(3, columns='DREB')
for i, (index, team) in enumerate(three_best_teams.iterrows()):
    print(index_to_strings(i, teams_dict[index], 'avg number of defensive rebounds', f'{team.values[0]:.2f}'))

In [ ]:
three_best_players = game_player_score.loc[:, ['PlayerID', 'DREB']].groupby(['PlayerID']).mean().nlargest(3, columns='DREB')
for i, (index, player) in enumerate(three_best_players.iterrows()):
    print(index_to_strings_players(i, players_dict[index], 'avg number of defensive rebounds', f'{player.values[0]:.2f}'))

### Asists

__Histogram of assits in a single game per group in the current season (2022-2023) so far__

In [ ]:
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(game_team_score['AST'].to_numpy(), alpha = 0.5, kde = True)
ax.set(title = "# of assits in a single game (per team)", xlabel = '# of assits', ylabel='count[games]')
plt.legend([],[], frameon = False)
plt.show(fig);

In [ ]:
three_best_teams = game_team_score.loc[:, ['TeamID', 'AST']].groupby(['TeamID']).mean().nlargest(3, columns='AST')
for i, (index, team) in enumerate(three_best_teams.iterrows()):
    print(index_to_strings(i, teams_dict[index], 'avg of assits', f'{team.values[0]:.2f}'))

### Blocks

__Histogram of blocks in a single game per group in the current season (2022-2023) so far__

In [ ]:
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(game_team_score['BLK'].to_numpy(), alpha = 0.5, kde = True, bins = np.arange(0, 18))
ax.set(title = "# of blocks in a single game (per team)", xlabel = '# of blocks', ylabel='count[games]')
plt.legend([],[], frameon = False)
plt.show(fig);

In [ ]:
three_best_teams = game_team_score.loc[:, ['TeamID', 'BLK']].groupby(['TeamID']).mean().nlargest(3, columns='BLK')
for i, (index, team) in enumerate(three_best_teams.iterrows()):
    print(index_to_strings(i, teams_dict[index], 'avg of blocks', f'{team.values[0]:.2f}'))

### Steals

__Histogram of steals in a single game per group in the current season (2022-2023) so far__

In [ ]:
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(game_team_score['STL'].to_numpy(), alpha = 0.5, kde = True, bins = 20)
ax.set(title = "# of steals in a single game (per team)", xlabel = '# of steals', ylabel='count[games]')
plt.legend([],[], frameon = False)
plt.show(fig);

### Turnovers

__Histogram of turnovers in a single game per group in the current season (2022-2023) so far__

In [ ]:
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(game_team_score['TO'].to_numpy(), alpha = 0.5, kde = True, bins = 20)
ax.set(title = "# of turnovers in a single game (per team)", xlabel = '# of turnovers', ylabel='count[games]')
plt.legend([],[], frameon = False)
plt.show(fig);

### Teams "Power Ranking"

In [ ]:
game_team_score_grouped = game_team_score.groupby(['TeamID']).mean().reset_index()

In [ ]:
teams_wins = {teamID: 0 for teamID in teams_dict.keys()}    

for i, row in finished_games.iterrows():        
    visitor_team_pts = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == row['VisitorTeamID']), 'PTS'].to_numpy()[0]
    home_team_pts = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == row['HomeTeamID']), 'PTS'].to_numpy()[0]
    
    if visitor_team_pts > home_team_pts:
       teams_wins[row['VisitorTeamID']] += 1

    else:
        teams_wins[row['HomeTeamID']] += 1

for key, value in teams_wins.items():
    game_team_score_grouped.loc[game_team_score_grouped['TeamID'] == key, 'wins'] = value

In [ ]:
game_team_score_grouped['TeamScore'] = 0.2 * game_team_score_grouped['wins'] + \
                                        0.2 * game_team_score_grouped['PTS'] + \
                                        0.2 * game_team_score_grouped['AST'] + \
                                        0.1 * game_team_score_grouped['STL'] + \
                                        0.1 * game_team_score_grouped['BLK'] + \
                                        0.2 * game_team_score_grouped['REB']

In [ ]:
three_best_teams = game_team_score_grouped.nlargest(3, columns='TeamScore')
for i, (index, team) in enumerate(three_best_teams.iterrows()):
    print(index_to_strings(i, teams_dict[team['TeamID']], 'overall score', ''))

### Players "Power Ranking"

In [ ]:
game_player_score_grouped = game_player_score.groupby(['PlayerID']).mean()
game_player_score_grouped['PlayerScore'] = 0.4 * game_player_score_grouped['PTS'] + \
                                            0.2 * game_player_score_grouped['AST'] + \
                                            0.1 * game_player_score_grouped['STL'] + \
                                            0.1 * game_player_score_grouped['BLK'] + \
                                            0.2 * game_player_score_grouped['REB']

In [ ]:
three_best_players = game_player_score_grouped.nlargest(3, columns='PlayerScore')
for i, (index, player) in enumerate(three_best_players.iterrows()):
    print(index_to_strings_players(i, players_dict[index], 'overall score', ''))

## Team level analysis
seleceted team - Phoenix Suns

In [ ]:
selected_team_ID = 1610612756
selected_team_games = finished_games.loc[(finished_games['HomeTeamID'] == selected_team_ID) | (finished_games['VisitorTeamID'] == selected_team_ID), :]

for i, row in selected_team_games.iterrows():
    # print(type(row['HomeTeamID']))
    if row['HomeTeamID'] == selected_team_ID:
        home = True
        selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'home'] = True
    else:
        home = False
        selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'home'] = False

    selected_team_pts = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'PTS'].to_numpy()[0]

    # Fill the df with stats from the game
    selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'PTS'] = selected_team_pts
    selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'FGPER'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'FGPER'].to_numpy()[0]
    selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'ThreePPER'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'ThreePPER'].to_numpy()[0]
    selected_team_games.loc[selected_team_games['ID'] == row['ID'], '3FGM_to_FGM'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), '3FGM_to_FGM'].to_numpy()[0]
    # selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'AST'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'AST'].to_numpy()[0]
    # selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'BLK'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'BLK'].to_numpy()[0]
    # selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'DREB'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'DREB'].to_numpy()[0]
    # selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'FGA'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'FGA'].to_numpy()[0]
    # selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'FGM'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'FGM'].to_numpy()[0]
    # selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'FTA'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'FTA'].to_numpy()[0]
    # selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'FTM'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'FTM'].to_numpy()[0]
    # selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'FTPER'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'FTPER'].to_numpy()[0]
    # selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'ThreePA'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'ThreePA'].to_numpy()[0]
    # selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'ThreePM'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'ThreePM'].to_numpy()[0]
    # selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'OREB'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'OREB'].to_numpy()[0]
    # selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'REB'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'REB'].to_numpy()[0]
    # selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'STL'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'STL'].to_numpy()[0]
    # selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'TO'] = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == selected_team_ID), 'TO'].to_numpy()[0]

    # print(selected_team_pts)

    if home:
        other_team_pts = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == row['VisitorTeamID']), 'PTS'].to_numpy()[0]
    else:
        other_team_pts = game_team_score.loc[(game_team_score['GameID'] == row['ID']) & (game_team_score['TeamID'] == row['HomeTeamID']), 'PTS'].to_numpy()[0]
    
    # print(other_team_pts)

    if other_team_pts > selected_team_pts:
        selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'result'] = 'L'
    else:
        selected_team_games.loc[selected_team_games['ID'] == row['ID'], 'result'] = 'W'


In [ ]:
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(selected_team_games['result'].to_numpy(), alpha = 0.5)
ax.set(title = "Game results this season", xlabel = 'result', ylabel='count[games]')
plt.legend([],[], frameon = False)
plt.show(fig);

In [ ]:
fig = plt.figure(figure_count, figsize = (9, 6))
figure_count += 1
ax = sns.histplot(selected_team_games.loc[selected_team_games['home'] == True, 'result'].to_numpy(), alpha = 0.5)
ax.set(title = "Game at home results this season", xlabel = 'result', ylabel='count[games]')
plt.legend([],[], frameon = False)
plt.show(fig);

In [ ]:
selected_team_games_copy = selected_team_games.to_pandas().set_index('Date')

line_plot = selected_team_games_copy.home

line_plot2 = selected_team_games_copy.result

bar_plot = line_plot.groupby([line_plot.index.year, line_plot.index.month, line_plot2]).count().unstack()

fig, ax = plt.subplots(num = figure_count)
figure_count += 1
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.17)
bar_plot.plot(ax = ax, kind='bar', figsize=(9, 6), colormap='coolwarm', alpha = 0.5)
ax.set_xlabel('Date of game [year, month]')
ax.set_ylabel('Count [games]')
ax.set_title('All games Win/Lose segmentation per month')
plt.show(fig);

In [ ]:
selected_team_games_at_home_copy = selected_team_games.loc[selected_team_games['home'] == True, :].to_pandas().set_index('Date')

line_plot = selected_team_games_at_home_copy.home

line_plot2 = selected_team_games_at_home_copy.result

bar_plot = line_plot.groupby([line_plot.index.year, line_plot.index.month, line_plot2]).count().unstack()

fig, ax = plt.subplots(num = figure_count)
figure_count += 1
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.17)
bar_plot.plot(ax = ax, kind='bar', figsize=(9, 6), colormap='coolwarm', alpha = 0.5)
ax.set_xlabel('Date of game [year, month]')
ax.set_ylabel('Count [games]')
ax.set_title('Home games Win/Lose segmentation per month')
plt.show(fig);

In [ ]:
line_plot = selected_team_games_copy.home

bar_plot = selected_team_games_copy['3FGM_to_FGM'].groupby([line_plot.index.year, line_plot.index.month]).mean()

fig, ax = plt.subplots(num = figure_count)
figure_count += 1
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.17)
bar_plot.plot(ax = ax, kind='bar', figsize=(9, 6), colormap='coolwarm', alpha = 0.5)
ax.set_xlabel('Date of game [year, month]')
ax.set_ylabel('Count [games]')
ax.set_title('3FGM out of FGM per month')
plt.show(fig);

In [ ]:
bar_plot = selected_team_games_copy['ThreePPER'].groupby([line_plot.index.year, line_plot.index.month]).mean()

fig, ax = plt.subplots(num = figure_count)
figure_count += 1
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.17)
bar_plot.plot(ax = ax, kind='bar', figsize=(9, 6), colormap='coolwarm', alpha = 0.5)
ax.set_xlabel('Date of game [year, month]')
ax.set_ylabel('Count [games]')
ax.set_title('3FG% per month')
plt.show(fig);

In [ ]:

grouped = pd.DataFrame(game_line_up.loc[game_line_up['TeamID'] == selected_team_ID, ['GameID', 'PlayerID']].to_pandas().groupby('GameID')['PlayerID'].apply(list))
for gameID in grouped.index:
    grouped.loc[gameID, 'result'] = selected_team_games.loc[selected_team_games['ID'] == gameID, 'result'].to_numpy()[0]

In [ ]:
grouped['PlayerID'] = grouped.PlayerID.apply(tuple)
bar_plot = grouped.reset_index().groupby(['PlayerID', 'result'])['GameID'].count().unstack()

fig, ax = plt.subplots(num = figure_count)
figure_count += 1
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.17)
bar_plot.plot(ax = ax, kind='bar', figsize=(9, 6), colormap='coolwarm', alpha = 0.5)
ax.set_xlabel('Players group')
ax.set_ylabel('Count [games]')
ax.set_title('Win/Lose segmentation according to players praticipating in the game')
ax.set_xticklabels(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F'])
plt.show(fig);

In [ ]:
switcher = {
        0: 'A',
        1: 'B',
        2: 'C',
        3: 'D',
        4: 'E',
        5: 'F'
    }
for i, players in enumerate(bar_plot.index):
    print(f'Players in group {switcher[i]}:')
    print([players_dict[player] for player in players])